In [1]:
import json
import pandas as pd
from IPython.display import display, Image
from typing import TypedDict

from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report

In [2]:
from lib.Sequential import Sequential
from lib.Layer import Dense

In [3]:
class JsonReader:
    def __init__(self, filename: str):
        self.filename = filename
        self.data = None

    def read(self):
        with open(self.filename, 'r') as f:
            self.data = json.load(f)
        return self.data

    def get(self, index: int|str):
        return self.data[index]

    def length(self):
        return len(self.data)
    
class Layers(TypedDict):
    number_of_neurons: int
    activation_function : str

class ModelData(TypedDict):
    input_size: int
    layers: Layers

class LearningParameters(TypedDict): 
    learning_rate: float
    batch_size: int
    max_iteration: int
    error_threshold: float

class Case(TypedDict):
    model: ModelData
    input: list[list[list[float]]]
    initial_weights: list[list[list[float]]]
    target: list[list[list[float]]]
    learning_parameters: LearningParameters
    
class Expect(TypedDict):
    stopped_by: str
    final_weights: list[list[list[float]]]
    
class ModelConfig(TypedDict):
    case: Case
    expect: Expect
    
class JsonParser:
    def parse_model_config(self, json_path: str) -> ModelConfig:
        model_config: ModelConfig = JsonReader(json_path).read()
        return model_config

In [4]:
class ModelFactory:
    def build(self, model_config: ModelConfig) -> Sequential:
        # Get the model case
        case = model_config["case"]
        layers = case["model"]
        weights = case["initial_weights"]
        inputs = case["input"]
        
        # Membangun model ANN
        model = Sequential()
        for i in range (len(layers['layers'])):
            layer = layers['layers'][i]
            weight = weights[i]
            dense_layer = Dense(layer["number_of_neurons"], activation=layer["activation_function"])
            dense_layer.build(weight)
            model.add(dense_layer)
        
        return model
    
    def load(self, path: str) -> Sequential:
        with open(path, 'rb') as f:
            return pickle.load(f)

In [5]:
class model_tester:
    @staticmethod
    def test(test_case: str):
        model_config: ModelConfig = JsonParser().parse_model_config(test_case)
        model_factory = ModelFactory()
        model = model_factory.build(model_config)

        case = model_config['case']
        learning_parameters = case["learning_parameters"]
        data = case['input']
        target = case['target']

        stop_reason = model.fit(
            data, 
            target, 
            epochs=learning_parameters['max_iteration'], 
            batch_size=learning_parameters['batch_size'], 
            learning_rate=learning_parameters['learning_rate'], 
            error_threshold=learning_parameters['error_threshold'], 
            random_state=42, 
            verbose=True
        )
        
        print("\n-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-")
        print("Model Summary")
        model.summary()
        print("Expected final weights:",
            model_config['expect'].get("final_weights"))

        print("\n-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-")
        print(stop_reason)

        print("Expected stop reason: ", model_config['expect']['stopped_by'])

In [6]:
model_name = 'test/linear.json'
model_tester.test(model_name)

Epoch 1/1
2/2 [==============================] - loss: 0.1108 - time: 0.0039 s

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
Model Summary
 Model: "sequential"
-----------------------------------------------
 Layer (type)        Output Shape       Param #
 dense (Dense)       (None, 3)          9      
Total params: 9

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
Layer Summary
Layer
activation = linear,
weights =
[[ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]],
bias = [0.22 0.36 0.11]
Expected final weights: [[[0.22, 0.36, 0.11], [0.64, 0.3, -0.89], [0.28, -0.7, 0.37]]]

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
[Stop] Maximum number of iteration reached.
Expected stop reason:  max_iteration


In [7]:
model_name = 'test/linear_small_lr.json'
model_tester.test(model_name)

Epoch 1/1
2/2 [==============================] - loss: 0.1108 - time: 0.0000 s

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
Model Summary
 Model: "sequential"
-----------------------------------------------
 Layer (type)        Output Shape       Param #
 dense (Dense)       (None, 3)          9      
Total params: 9

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
Layer Summary
Layer
activation = linear,
weights =
[[ 0.4024  0.201  -0.7019]
 [ 0.1018 -0.799   0.4987]],
bias = [0.1012 0.3006 0.1991]
Expected final weights: [[[0.1008, 0.3006, 0.1991], [0.402, 0.201, -0.7019], [0.101, -0.799, 0.4987]]]

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
[Stop] Maximum number of iteration reached.
Expected stop reason:  max_iteration


In [8]:
model_name = 'test/linear_two_iteration.json'
model_tester.test(model_name)

Epoch 1/2
2/2 [==============================] - loss: 0.1108 - time: 0.0000 s
Epoch 2/2
2/2 [==============================] - loss: 0.0303 - time: 0.0000 s

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
Model Summary
 Model: "sequential"
-----------------------------------------------
 Layer (type)        Output Shape       Param #
 dense (Dense)       (None, 3)          9      
Total params: 9

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
Layer Summary
Layer
activation = linear,
weights =
[[ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]],
bias = [0.166 0.338 0.153]
Expected final weights: [[[0.166, 0.338, 0.153], [0.502, 0.226, -0.789], [0.214, -0.718, 0.427]]]

-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-=x=-
[Stop] Maximum number of iteration reached.
Expected

In [ ]:
# Load dataset iris
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = pd.Series(iris.target)
df

In [ ]:
df['target'].value_counts()

In [ ]:
# Melakukan splitting data train dan validation
X_train, X_val, y_train, y_val = train_test_split(df[iris.feature_names], df['target'], test_size=0.3, random_state=42, stratify=df['target'])

In [ ]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

In [ ]:
# Membangun model ANN
model = Sequential()
model.add(Dense(4, activation='sigmoid', input_shape=(X_train.shape[1],)))
model.add(Dense(6, activation='sigmoid', input_shape=(4,)))
model.add(Dense(3, activation='linear', input_shape=(6,)))

In [ ]:
# Fit model, menjalankan 1000 epochs
model.fit(X_train, y_train, epochs=1000, batch_size=4, learning_rate=0.1, error_threshold=0.3, random_state=42, verbose=True)

In [ ]:
model.summary()

In [ ]:
# Melakukan visualisasi model struktur jaringan
model.visualize()

# Mengambil gambar hasil visualisasi model untuk ditampilkan
image_path = "output/ffnn_graph.png"
display(Image(filename=image_path))

In [ ]:
# Melakukan prediksi dan melihat hasil prediksi
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

In [ ]:
# Menyimpan model yang telah dibuat dalam json
model.save("iris_model")